In [1]:
#!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
#!unzip ml-latest-small.zip

!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

--2020-02-08 18:42:24--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.6MB/s    in 0.3s    

2020-02-08 18:42:24 (15.6 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [0]:
import pandas as pd
import torch as torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter
#from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px
import math

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

,movieId,title,date,N,url,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:

ratings=pd.read_csv('/content/ml-100k/u.data',sep='\t',header=None,names=['userId','movieId','rating','timestamp'])
movies=pd.read_csv('/content/ml-100k/u.item',sep='|',header=None,encoding='latin-1',
                   names=['movieId', 'title', 'date', 'N', 'url', *[f'g{i}' for i in range(19)]])
ratings=ratings.join(movies.set_index('movieId'),on='movieId')
ratings['userId'] = ratings['userId'].astype('category')
ratings['movieId'] = ratings['movieId'].astype('category')

In [55]:
ratings

,userId,movieId,rating,timestamp,title,date,N,url,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15,g16,g17,g18
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-11...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,880175444,"First Wives Club, The (1996)",14-Sep-1996,NaN,http://us.imdb.com/M/title-exact?First%20Wives...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
99996,716,204,5,879795543,Back to the Future (1985),01-Jan-1985,NaN,http://us.imdb.com/M/title-exact?Back%20to%20t...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
99997,276,1090,1,874795795,Sliver (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Sliver%20(1993),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
99998,13,225,2,882399156,101 Dalmatians (1996),27-Nov-1996,NaN,http://us.imdb.com/M/title-exact?101%20Dalmati...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# save mappings of categorical id's
userId_map = dict(zip(ratings.userId.cat.codes, ratings.userId))
movieId_map = dict(zip(ratings.movieId.cat.codes, ratings.movieId))

In [0]:
data_x = torch.stack([torch.tensor(ratings['userId'].cat.codes.values),
                      torch.tensor(ratings['movieId'].cat.codes.values)],
                     axis=1).long()
data_y = torch.tensor(ratings['rating'].values).float()
dataset = data.TensorDataset(data_x, data_y)

In [0]:
bs=128
train_n = int(len(dataset)*0.9)
valid_n = len(dataset) - train_n
splits = [train_n,valid_n]
assert sum(splits) == len(dataset)
trainset,devset = torch.utils.data.random_split(dataset,splits)
train_dataloader = data.DataLoader(trainset,batch_size=bs,shuffle=True)
dev_dataloader = data.DataLoader(devset,batch_size=bs,shuffle=True)

In [0]:
# copied from fastai: 
def trunc_normal_(x, mean=0., std=1.):
    "Truncated normal initialization."
    # From https://discuss.pytorch.org/t/implementing-truncated-normal-initializer/4778/12
    return x.normal_().fmod_(2).mul_(std).add_(mean)

In [0]:
class NNet(nn.Module):
  def __init__(self, num_movies, num_users, embeddings_dim):
    super().__init__()
    self.movie_embeddings = nn.Embedding(num_movies,embeddings_dim)
    self.movie_bias = nn.Embedding(num_movies,1)
    self.user_embeddings = nn.Embedding(num_users,embeddings_dim)
    self.user_bias = nn.Embedding(num_movies,1)

    # See https://arxiv.org/abs/1711.09160
    with torch.no_grad(): trunc_normal_(self.movie_embeddings.weight, std=0.01)
    with torch.no_grad(): trunc_normal_(self.user_embeddings.weight, std=0.01)
    with torch.no_grad(): trunc_normal_(self.user_bias.weight, std=0.01)
    with torch.no_grad(): trunc_normal_(self.movie_bias.weight, std=0.01)

  def forward(self, X):
    user_emb = self.user_embeddings(X[:,0])
    movie_emb = self.movie_embeddings(X[:,1])
    # sigmoind - Jeremy's trick for getting better results
    return torch.sigmoid((user_emb*movie_emb).sum(1) + self.user_bias(X[:,0]).squeeze() + self.movie_bias(X[:,1]).squeeze())*5.5
    


In [0]:
# fit/test functions
def fit(iterator, model, optimizer, criterion):
    train_loss = 0
    model.train()
    for x,y in iterator:
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += loss.item()*x.shape[0]
        loss.backward()
        optimizer.step()
    return train_loss / len(iterator.dataset)

def test(iterator, model, criterion):
    train_loss = 0
    model.eval()
    for x,y in iterator:                    
        with torch.no_grad():
            y_hat = model(x)
        loss = criterion(y_hat, y)
        train_loss += loss.item()*x.shape[0]
    return train_loss / len(iterator.dataset)


In [0]:
def train_n_epochs(model, n, optimizer):

    criterion = nn.MSELoss().to(device)
    for epoch in range(n):
        start_time = time.time()
        train_loss = fit(train_dataloader, model, optimizer, criterion)
        valid_loss = test(dev_dataloader, model, criterion)
        secs = int(time.time() - start_time)
        print(f'Epoch: {epoch} time: {secs}[s]')
        print(f'\tTrain Loss: {(train_loss):.4f}')
        print(f'\tValidation Loss: {(valid_loss):.4f}')

In [63]:
model = NNet(len(movieId_map),len(userId_map),40)
wd=1e-4
lr=0.001
epochs=10
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
train_n_epochs(model,epochs,optimizer)

Epoch: 0 time: 2[s]
	Train Loss: 1.3085
	Validation Loss: 0.9365
Epoch: 1 time: 2[s]
	Train Loss: 0.8986
	Validation Loss: 0.8734
Epoch: 2 time: 2[s]
	Train Loss: 0.8331
	Validation Loss: 0.8422
Epoch: 3 time: 2[s]
	Train Loss: 0.7926
	Validation Loss: 0.8275
Epoch: 4 time: 2[s]
	Train Loss: 0.7615
	Validation Loss: 0.8172
Epoch: 5 time: 3[s]
	Train Loss: 0.7307
	Validation Loss: 0.8066
Epoch: 6 time: 2[s]
	Train Loss: 0.6981
	Validation Loss: 0.7992
Epoch: 7 time: 2[s]
	Train Loss: 0.6625
	Validation Loss: 0.7940
Epoch: 8 time: 2[s]
	Train Loss: 0.6254
	Validation Loss: 0.7901
Epoch: 9 time: 2[s]
	Train Loss: 0.5869
	Validation Loss: 0.7911


Check out specific embeddings

In [69]:
# cluster movies
df = pd.DataFrame(ratings.groupby(['title','movieId'])['rating'].count()).reset_index().sort_values(by=['rating'],ascending=False).head(100)
X = model.movie_embeddings(torch.tensor(df['movieId'].cat.codes.values).long().unsqueeze(0)).squeeze(0).detach()
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
df['cluster'] = kmeans.labels_
df

,title,movieId,rating,cluster
1413,Star Wars (1977),50,583,3
337,Contact (1997),258,509,3
505,Fargo (1996),100,508,1
1248,Return of the Jedi (1983),181,507,3
872,Liar Liar (1997),294,485,0
...,...,...,...,...
36,Aladdin (1992),95,219,4
1606,Volcano (1997),678,219,0
1524,To Kill a Mockingbird (1962),427,219,2
109,Babe (1995),8,219,4


In [0]:
#ldr = TSNE(n_components=3, random_state=0)
ldr = PCA(n_components=3)
Y = ldr.fit_transform(X)

In [0]:
df['x'] = Y[:, 0]
df['y'] = Y[:, 1]
df['z'] = Y[:, 2]

In [0]:
df['cluster'] = df['cluster'].apply(str)

In [73]:
fig = px.scatter_3d(df.head(100), x='x', y='y', z='z',hover_name='title',
              color='cluster')
fig.show()